# Main Work Notebook

In [1]:
# System audio I/O backend
# !sudo apt-get update && sudo apt-get install -y ffmpeg

# Python deps
!pip install silero-vad

[sudo] password for sharjeel: 
sudo: a password is required
^C
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 2.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [silero-vad]5 [onnxruntime]


In [ ]:
!python trim_with_silero_vad.py --in_dir ./Duaa --out_dir wavs --keep_silence 0.25 --viz_dir data/viz

ok          Female_ID_41457967_10.wav
ok          Female_ID_41457967_103.wav
ok          Female_ID_41457967_107.wav
ok          Female_ID_41457967_11.wav
ok          Female_ID_41457967_12.wav
ok          Female_ID_41457967_13.wav
ok          Female_ID_41457967_15.wav
ok          Female_ID_41457967_16.wav
ok          Female_ID_41457967_17.wav
ok          Female_ID_41457967_172.wav
ok          Female_ID_41457967_173.wav
ok          Female_ID_41457967_174.wav
ok          Female_ID_41457967_175.wav
ok          Female_ID_41457967_176.wav
ok          Female_ID_41457967_177.wav
ok          Female_ID_41457967_178.wav
ok          Female_ID_41457967_179.wav
ok          Female_ID_41457967_18.wav
ok          Female_ID_41457967_180.wav
ok          Female_ID_41457967_181.wav
ok          Female_ID_41457967_182.wav
ok          Female_ID_41457967_183.wav
ok          Female_ID_41457967_184.wav
ok          Female_ID_41457967_185.wav
ok          Female_ID_41457967_186.wav
ok          Female_ID_41457967_18